In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.cost_function import SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.genetic_algorithm.simple_genetic_algorithm import (
    SimpleGeneticAlgorithm,
    SimpleGeneticAlgorithmParameters,
)
from seapopym_optimization.genetic_algorithm.simple_logbook import Logbook
from seapopym_optimization.model_generator import NoTransportModelGenerator


In [2]:
import logging

logging.basicConfig(level=logging.WARNING, format="\n%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("seapopym_optimization")
logger.setLevel(logging.DEBUG)

In [3]:
fg1 = NoTransportFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

fg2 = NoTransportFunctionalGroup(
    name="FG2",
    day_layer=1,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

In [4]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_transfert", "FG2_energy_transfert"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [5]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)

In [6]:
fg_set.unique_functional_groups_parameters_ordered().keys()

dict_keys(['FG1_energy_transfert', 'lambda_temperature_0', 'gamma_lambda_temperature', 'tr_0', 'gamma_tr', 'FG2_energy_transfert'])

In [7]:
parameters_set = [1, np.inf, 1, 1, -1, 1]
functional_group_parameters = fg_set.generate(parameters_set)
functional_group_parameters

[{'day_layer': 0,
  'night_layer': 0,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1},
 {'day_layer': 1,
  'night_layer': 1,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1}]

In [8]:
nb_days = 360
temperature = xr.DataArray(
    data=np.zeros((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=np.ones((nb_days, 1, 1)),
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "kg/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
observation = xr.DataArray(
    data=np.ones((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "layer"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "layer": [0, 1],
    },
    attrs={
        "units": "kg/m^2",
    },
)
observation.time.attrs = {"axis": "T"}
temperature.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}

observation.latitude.attrs = {"axis": "Y"}
temperature.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}

observation.longitude.attrs = {"axis": "X"}
temperature.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

observation.layer.attrs = {"axis": "Z"}
temperature.depth.attrs = {"axis": "Z"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
)
with xr.set_options(keep_attrs=True):
    observation_2 = observation.sel(layer=[1]) / 2

In [9]:
observation_1 = TimeSeriesObservation(
    name="Observation_all",
    observation=observation,
)
observation_2 = TimeSeriesObservation(
    name="Observation_meso",
    observation=observation_2,
)

In [10]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameter)

In [11]:
# model = model_generator.generate(functional_group_parameters=functional_group_parameters)

In [12]:
# model.state

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[observation_1, observation_2],
    functional_groups=[fg1, fg2],
    root_mse=True,
    centered_mse=False,
    normalized_mse=False,
)
# partial_cost_function = cost_function.generate()
# partial_cost_function(parameters_set)


In [14]:
metaparam = SimpleGeneticAlgorithmParameters(
    ETA=4, INDPB=0.2, CXPB=0.5, MUTPB=0.2, NGEN=3, POP_SIZE=10, cost_function_weight=(-1, -1)
)
genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam, cost_function=cost_function, constraint=[constraint]
)

In [15]:
viewer = genetic_algorithm.optimize()

In [16]:
genetic_algorithm.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.903213   
                                       1                      0.306994   
                                       2                      0.170091   
                                       3                      0.668542   
                                       4                      0.156630   
                                       5                      0.246708   
                                       6                      0.884615   
                                       7                      0.018091   
                                       8                      0.860325   
                                       9                      0.346342   
1          False                       0                      0.346342   
                                       1                      0.170091   
                                       2                      0.828471   
           True                        3                      0.246708   
                                       4                      0.246708   
                                       5                      0.018091   
           False                       6                      0.156630   
                                       7                      0.246708   
           True                        8                      0.306994   
                                       9                      0.346342   
2          True                        0                      0.246708   
                                       1                      0.306994   
                                       2                      0.156630   
                                       3                      0.346342   
           False                       4                      0.346342   
                                       5                      0.246708   
           True                        6                      0.246708   
                                       7                      0.156630   
           False                       8                      0.156630   
                                       9                      0.246708   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     40.069842   
                                       1                      0.844357   
                                       2                      9.852331   
                                       3                     15.180335   
                                       4                     48.529656   
                                       5                     50.260537   
                                       6                     82.303120   
                                       7                     94.766848   
                                       8                     35.475724   
                                       9                     85.056469   
1          False                       0                      9.852331   
                                       1                     85.056469   
                                       2                     35.475724   
           True                        3                     50.260537   
                                       4                     50.260537   
                                       5                     94.766848   
           False                       6                     48.529656   
                                       7                     50.260537   
   

In [17]:
# unevaluated_logbook = genetic_algorithm.logbook.loc[[0]].copy()
# unevaluated_logbook

In [18]:
# unevaluated_logbook.iloc[:5, -3:] = np.nan
# unevaluated_logbook = Logbook(unevaluated_logbook)
# unevaluated_logbook

In [19]:
# new_genetic_algorithm = SimpleGeneticAlgorithm(
#     meta_parameter=metaparam,
#     cost_function=cost_function,
#     constraint=[constraint],
#     logbook=unevaluated_logbook,
# )


In [20]:
# viewer = new_genetic_algorithm.optimize()

In [21]:
viewer.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.903213   
                                       1                      0.306994   
                                       2                      0.170091   
                                       3                      0.668542   
                                       4                      0.156630   
                                       5                      0.246708   
                                       6                      0.884615   
                                       7                      0.018091   
                                       8                      0.860325   
                                       9                      0.346342   
1          False                       0                      0.346342   
                                       1                      0.170091   
                                       2                      0.828471   
           True                        3                      0.246708   
                                       4                      0.246708   
                                       5                      0.018091   
           False                       6                      0.156630   
                                       7                      0.246708   
           True                        8                      0.306994   
                                       9                      0.346342   
2          True                        0                      0.246708   
                                       1                      0.306994   
                                       2                      0.156630   
                                       3                      0.346342   
           False                       4                      0.346342   
                                       5                      0.246708   
           True                        6                      0.246708   
                                       7                      0.156630   
           False                       8                      0.156630   
                                       9                      0.246708   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     40.069842   
                                       1                      0.844357   
                                       2                      9.852331   
                                       3                     15.180335   
                                       4                     48.529656   
                                       5                     50.260537   
                                       6                     82.303120   
                                       7                     94.766848   
                                       8                     35.475724   
                                       9                     85.056469   
1          False                       0                      9.852331   
                                       1                     85.056469   
                                       2                     35.475724   
           True                        3                     50.260537   
                                       4                     50.260537   
                                       5                     94.766848   
           False                       6                     48.529656   
                                       7                     50.260537   
   

In [22]:
viewer.stats()

,mean,std,min,max,valid,from_previous_generation
Generation,,,,,,
0,-0.428362,0.038994,-0.496484,-0.398867,5,0.000000
1,-0.420889,0.032383,-0.496484,-0.387085,8,0.500000
2,-0.410846,0.012663,-0.421084,-0.387085,9,0.555556


In [23]:
viewer.hall_of_fame()

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
1          False                       0                      0.346342   
2          False                       4                      0.346342   
0          False                       2                      0.170091   
1          False                       1                      0.170091   
0          False                       7                      0.018091   
2          False                       5                      0.246708   
0          False                       5                      0.246708   
1          False                       6                      0.156630   
                                       7                      0.246708   
0          False                       4                      0.156630   
                                       9                      0.346342   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
1          False                       0                      9.852331   
2          False                       4                      9.852331   
0          False                       2                      9.852331   
1          False                       1                     85.056469   
0          False                       7                     94.766848   
2          False                       5                     50.260537   
0          False                       5                     50.260537   
1          False                       6                     48.529656   
                                       7                     50.260537   
0          False                       4                     48.529656   
                                       9                     85.056469   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
1          False                       0                          0.767231   
2          False                       4                          0.767231   
0          False                       2                          0.767231   
1          False                       1                          0.035027   
0          False                       7                          0.315206   
2          False                       5                          0.722269   
0          False                       5                          0.722269   
1          False                       6                          0.722269   
                                       7                          0.143593   
0          False                       4                          0.143593   
                                       9                          0.035027   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
1          False                       0           2.186004 -0.834442   
2          False                       4           6.554599 -0.831054   
0          False                       2           2.186004 -0.782788   
1          False                       1           4.451139 -0.782788   
0          False                       7           1.070838 -0.037770   
2          False                       5           2.186004 -0.834442   
0          False                       5           6.554599 -0.831054   
1          False                       6           6.554599 -0.410299   
                              

In [24]:
viewer.fitness_evolution(points="all")

In [25]:
viewer.box_plot(3)

In [26]:
viewer.parallel_coordinates()[0]

In [27]:
viewer.parameters_standardized_deviation()

In [28]:
viewer.shannon_entropy(bins=100_000)

In [30]:
for fig in viewer.time_series(10):
    fig.show()

0 <xarray.DataArray 'layer' (layer: 2)> Size: 16B
array([0, 1])
Coordinates:
  * layer    (layer) int64 16B 0 1
Attributes:
    axis:     Z
1 <xarray.DataArray 'layer' (layer: 2)> Size: 16B
array([0, 1])
Coordinates:
  * layer    (layer) int64 16B 0 1
Attributes:
    axis:     Z
1 <xarray.DataArray 'layer' (layer: 1)> Size: 8B
array([1])
Coordinates:
  * layer    (layer) int64 8B 1
Attributes:
    axis:     Z
